# aggregate results

0.997706276413
0.997421607422
0.996728503538
0.997075149144
0.998020443775


In [1]:
import numpy as np
import pandas as pd
import xgboost
from sklearn.metrics import roc_auc_score

/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
try:
    hn = pd.read_hdf("kernel.h5", "train")
except:
    print("kernel.h5 not found")
    hough = False
else:
    hough = True
    ht = pd.read_hdf("kernel.h5", "test")
    hfea = list(hn.columns)

In [3]:
dl = [pd.read_csv('DS_2_train.csv', index_col=0)]
# dl.append(pd.read_hdf("data-r1.h5", "train"))
dl.append(pd.read_hdf("data-r1.h5", "train"))
for i in range(2,7):
    dl.append(pd.read_hdf("data-r{}.h5".format(i), "train"))
for i in range(6):
    dl.append(pd.read_hdf("hiss-r{}.h5".format(i), "train"))
d0 = pd.concat(dl, axis=1)
print(d0.shape)

/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(9806386, 27)


In [4]:
el = [pd.read_csv('DS_2_test.csv', index_col=0)]
el.append(pd.read_hdf("data-r1.h5", "test"))
for i in range(2,7):
    el.append(pd.read_hdf("data-r{}.h5".format(i), "test"))
for i in range(6):
    el.append(pd.read_hdf("hiss-r{}.h5".format(i), "test"))
e0 = pd.concat(el, axis=1)
print(e0.shape)

/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(9807599, 25)


In [5]:
if hough:
    d0 = pd.concat([d0, hn], axis=1)
    e0 = pd.concat([e0, ht], axis=1)
print(d0.shape, e0.shape)

(9806386, 37) (9807599, 35)


In [6]:
print(e0.columns, d0.columns)

Index(['X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'brick_number', 'dfore', 'dback',
       'dfore2', 'dback2', 'dfore3', 'dback3', 'dfore4', 'dback4', 'dfore5',
       'dback5', 'dfore6', 'dback6', 'cnt0', 'cnt1', 'cnt2', 'cnt3', 'cnt4',
       'cnt5', 'hb0', 'hb1', 'hb2', 'hb4', 'hb8', 'rb0', 'rb1', 'rb2', 'rb4',
       'rb8'],
      dtype='object') Index(['event_id', 'X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'signal', 'brick_number',
       'dfore', 'dback', 'dfore2', 'dback2', 'dfore3', 'dback3', 'dfore4',
       'dback4', 'dfore5', 'dback5', 'dfore6', 'dback6', 'cnt0', 'cnt1',
       'cnt2', 'cnt3', 'cnt4', 'cnt5', 'hb0', 'hb1', 'hb2', 'hb4', 'hb8',
       'rb0', 'rb1', 'rb2', 'rb4', 'rb8'],
      dtype='object')


In [7]:
print(e0.head(), d0.head())

                  X             Y        Z        TX        TY      chi2  \
index                                                                      
0      56472.238281  54979.070312  19395.0 -0.161979 -0.471001  2.681201   
1      70306.289062  42301.472656  37497.0 -0.205394 -0.042299  1.413991   
2      30427.056641  30543.578125  42669.0  0.234635  0.114081  2.702541   
3      45467.257812  27151.515625  71115.0 -0.319327  0.561254  0.593076   
4      59172.652344  33032.457031  41376.0  0.467150 -0.332887  2.236792   

       brick_number         dfore         dback        dfore2    ...     \
index                                                            ...      
0                68  4.641073e+05  3.821470e+06  2.056737e+06    ...      
1                28  1.545539e+06  7.463376e+05  3.574570e+06    ...      
2                34  1.195922e+06  1.540890e+06  3.068504e+05    ...      
3                28  3.642606e+05  1.594315e+06  1.000000e+06    ...      
4                

In [25]:
# e0[ofeature].to_hdf("shared.h5", "test", complevel=9)
# hough = False


In [10]:
features = ["X", "Y", "Z", "TX", "TY", "chi2"] + ["dfore{}".format(i) for i in range(2,7)]
features += ["dback{}".format(i) for i in range(2,7)]
features += ["dback", "dfore"]
features += ["cnt{}".format(i) for i in range(4)]
if hough:
    features += hfea
print(features)    

['X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'dfore2', 'dfore3', 'dfore4', 'dfore5', 'dfore6', 'dback2', 'dback3', 'dback4', 'dback5', 'dback6', 'dback', 'dfore', 'cnt0', 'cnt1', 'cnt2', 'cnt3', 'hb0', 'hb1', 'hb2', 'hb4', 'hb8', 'rb0', 'rb1', 'rb2', 'rb4', 'rb8']


In [48]:
N = 5
# xgb1 = xgboost.XGBClassifier(n_estimators=500, nthread=18, max_depth=4, reg_alpha=0, reg_lambda=0.5)
xgb1 = xgboost.XGBClassifier(n_estimators=1000, objective="binary:logistic", nthread=18, max_depth=4, learning_rate=0.05, reg_alpha=0, reg_lambda=0.5, min_child_weight = 1, scale_pos_weight = 1)
xgbi = xgboost.XGBClassifier(n_estimators=1000, objective="binary:logistic", nthread=18, max_depth=4, learning_rate=0.05, reg_alpha=0.01, reg_lambda=0.5, min_child_weight = 1, scale_pos_weight = 1)
def cvh(i):
    test = d0.query("brick_number >= {} & brick_number < {}".format(i, i+100//N))
    train = d0.query("brick_number < {} | brick_number >= {}".format(i, i+100//N))
    train_s = train.query("signal==1")
    train1 = pd.concat((train_s, train.query("signal==0").sample(len(train_s))))

    xgb1.fit(train1[features], train1.signal)
    pred0 = xgb1.predict_proba(test[features])[:,1]
    print(roc_auc_score(test.signal, pred0))
    return pd.DataFrame({'rst0': pred0}, index=test.index)

In [41]:
if hough:
    i = 41
    test = d0.query("brick_number >= {} & brick_number < {}".format(i, i+100//N))
    train = d0.query("brick_number < {} | brick_number >= {}".format(i, i+100//N))
    train_s = train.query("signal==1")
    train1 = pd.concat((train_s, train.query("signal==0").sample(len(train_s))))

    xgb1.fit(train1[features], train1.signal)
    pred0 = xgb1.predict_proba(test[features])[:,1]
    print(roc_auc_score(test.signal, pred0))
else:
    hpdn = pd.concat([cvh(i) for i in range(1, 101, 100//N)])
    print(len(hpdn), len(d0))
    hpdn.head()


0.998500675949


In [47]:
xgb1.fit(train[features], train.signal)
pred0 = xgb1.predict_proba(test[features])[:,1]
print(roc_auc_score(test.signal, pred0))


0.998526290623


In [ ]:
xgbi.fit(train[features], train.signal)
pred0 = xgbi.predict_proba(test[features])[:,1]
print(roc_auc_score(test.signal, pred0))


max_depth=4: 0.998502320271
max_depth=6: 0.998447495776
learning_rate=0.1: 0.998500675949
learning_rate=0.05: 0.99850024116
n_estimators=1000: 0.998505775939
imbalanced: 0.998526290623

In [29]:
d0_s = d0.query("signal==1")
d01 = pd.concat((d0_s, d0.query("signal==0").sample(len(d0_s))))
print(len(d0_s), len(d01))

156950 313900


In [14]:
# pred0 = xgb1.predict_proba(test[features])[:,1]
# print(roc_auc_score(test.signal, pred0))

NameError: name 'test' is not defined

In [11]:
# xgb2 = xgboost.XGBClassifier(n_estimators=500, nthread=18, max_depth=4, reg_lambda=0.5)
xgb2 = xgboost.XGBClassifier(n_estimators=1000, objective="binary:logistic", nthread=18, max_depth=4, learning_rate=0.05, reg_alpha=0.01, reg_lambda=0.5, min_child_weight = 1, scale_pos_weight = 1)
xgb2.fit(d0[features], d0.signal)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=18,
       objective='binary:logistic', reg_alpha=0.01, reg_lambda=0.5,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [12]:
pred = xgb2.predict_proba(e0[features])[:,1]
if not hough:
    hpd = pd.DataFrame({'rst0': pred}, index=e0.index)

In [33]:
if not hough:
    hpd.to_hdf("rst0.h5", "test")
    hpdn.to_hdf("rst0.h5", "train")

In [13]:
def tresult(prediction, rev):
    baseline = pd.DataFrame(prediction, columns=['Prediction'])
    baseline.index.name = 'Id'
    baseline.to_csv('{}.csv.gz'.format(rev), header=True, compression="gzip")
if hough:
    tresult(pred, "h-2p-r1")
else:
    tresult(pred, "noratio")
    print("Now proceed to histogram4d.ipynb... Then reload")

In [27]:
mn = pd.read_hdf("mb.h5", "train")

mfea = list(filter(lambda x: x.find("Fwd")==0 or x.find("Bwd")==0, list(mn.columns)))

mt = pd.read_hdf("mb.h5", "test")

d1 = pd.concat([d0, mn], axis=1)

e1 = pd.concat([e0, mt], axis=1)

ff = features + mfea

In [28]:
xgbf = xgboost.XGBClassifier(n_estimators=500, nthread=18, max_depth=4, reg_alpha=0, reg_lambda=0.5)

In [30]:
i = 41
ftest = d1.query("brick_number >= {} & brick_number < {}".format(i, i+100//5))
ftrain = d1.query("brick_number < {} | brick_number >= {}".format(i, i+100//5))

In [31]:
ftrain_s = ftrain.query("signal==1")
ftrain1 = pd.concat((ftrain_s, ftrain.query("signal==0").sample(len(ftrain_s))))


In [32]:
xgbf.fit(ftrain1[ff], ftrain1.signal)
pred0 = xgb1.predict_proba(ftest[ff])[:,1]
print(roc_auc_score(ftest.signal, pred0))

ValueError: bad input shape (262054, 2)

In [46]:
np.sum(np.abs(d0.event_id - mn.event_id))

6962742516

In [44]:
mn.columns

Index(['event_id', 'X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'signal', 'brick_number',
       'X_n', 'Y_n', 'R_n', 'Phi_n', 'n_brick_trk', 'Fwd1_n', 'Fwd1_dTX',
       'Fwd1_dTY', 'Fwd1_sTX', 'Fwd1_sTY', 'Fwd1_chi2', 'Fwd1_s_chi2',
       'Fwd2_n', 'Fwd2_dTX', 'Fwd2_dTY', 'Fwd2_sTX', 'Fwd2_sTY', 'Fwd2_chi2',
       'Fwd2_s_chi2', 'Fwd3_n', 'Fwd3_dTX', 'Fwd3_dTY', 'Fwd3_sTX', 'Fwd3_sTY',
       'Fwd3_chi2', 'Fwd3_s_chi2', 'Bwd1_n', 'Bwd1_dTX', 'Bwd1_dTY',
       'Bwd1_sTX', 'Bwd1_sTY', 'Bwd1_chi2', 'Bwd1_s_chi2', 'Bwd2_n',
       'Bwd2_dTX', 'Bwd2_dTY', 'Bwd2_sTX', 'Bwd2_sTY', 'Bwd2_chi2',
       'Bwd2_s_chi2', 'Bwd3_n', 'Bwd3_dTX', 'Bwd3_dTY', 'Bwd3_sTX', 'Bwd3_sTY',
       'Bwd3_chi2', 'Bwd3_s_chi2', 'n_brick_tracks'],
      dtype='object')